## Imports

In [9]:
import sys
import os
import json
import glob
import subprocess
from collections import OrderedDict
import itertools
import numpy as np
import matplotlib.pyplot as plt

import librosa
from IPython.display import Audio as ipy_audio
from IPython.core.display import display



In [10]:
SCRIPT_DIR = os.path.dirname(os.path.abspath(""))
sys.path.append(SCRIPT_DIR)

from quicktranscribe import tonic, pitch, wave, kde
from mogra import tonnetz
from mogra.datatypes import Swar, normalize_frequency, ratio_to_swar, SWAR_BOUNDARIES

In [11]:
# syntonic comma in the 0 to 1 scale
SYNTONIC_COMMA = (librosa.hz_to_midi(220*81/80) - librosa.hz_to_midi(220))/12

## Util Functions

In [3]:
def fetch_audio(ra, savedir):
    for raag, vv in ra.items():
        for artist, url in vv.items():
            command = f"/opt/homebrew/bin/yt-dlp {url} -f 'ba' -x --audio-format 'mp3' --ffmpeg-location /opt/homebrew/bin/ffmpeg -P {savedir}/ -o {raag}-{artist}.mp3"
            result = subprocess.run(command, shell=True, capture_output=True)
            print(result.stdout.decode())
            if len(result.stderr) > 0:
                print("Error:", result.stderr.decode())

In [4]:
def annotate_tonic(track_path, plot=False):
    DEFAULT_TONIC = 220
    np.set_printoptions(suppress=True)
    
    start=7*60
    end=8*60
    y_stereo, sr = wave.read_audio_section(track_path + ".mp3", start, end)
    y_sample = librosa.to_mono(y_stereo.T)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=DEFAULT_TONIC)
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    if plot:
        plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
        plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    
    display(ipy_audio(y_sample, rate=sr))
    input("hear the audio and press any key to continue")
    
    peaks = sorted(peaks, key=lambda x: kde_sample[x], reverse=True)
    found_tonic = False
    for peak in peaks:
        # generate a sine wave of the peak frequency and play it
        fpeak = librosa.midi_to_hz(librosa.hz_to_midi(DEFAULT_TONIC) + 12 * peak / len(kde_sample))
        ypeak = librosa.tone(fpeak, duration=3)
        display(ipy_audio(ypeak, rate=sr))
        ft = input("Is this the tonic? (y/n): ")
        if ft == "y":
            found_tonic = True
            break
    
    if not found_tonic:
        print("No tonic found")
        return None
    
    # write tonic to file
    tonic.write_tonic(track_path + ".ctonic.txt", fpeak)

In [5]:
def read_sample_and_tonic(track_path):
    
    ctonic = tonic.read_tonic(track_path + ".ctonic.txt")
    # metadata = tonic.read_metadata(track_path + ".json")
    # pitch_annotations, aps = pitch.read_pitch(track_path + ".pitch.txt")
    
    # # full audio
    # y_sample, sr = wave.get_audio(track_path + ".mp3")

    # # 3-minute sample
    start=5*60
    end=8*60
    y_stereo, sr = wave.read_audio_section(track_path + ".mp3", start, end)
    y_sample = librosa.to_mono(y_stereo.T)
    # ipy_audio(data=y_sample, rate=sr)
    
    return y_sample, sr, ctonic

## Raags & Tracks

In [6]:
DATA_DIR = "concrete-demo/"

In [ ]:
raag_theoretical = {
    "Bhoop": [1, 10/9, 5/4, 3/2, 5/3],
    "Deshkar": [1, 9/8, 81/64, 3/2,  27/16],
    "Yaman": [1, 9/8, 5/4, 45/32, 3/2, 27/16, 15/8],
}
for raag, vv in raag_theoretical.items():
    raag_theoretical[raag] = [normalize_frequency(f) for f in vv]
    print(raag)
    for ii in range(len(raag_theoretical[raag])):
        print(f"{ratio_to_swar(raag_theoretical[raag][ii])}: {np.round(raag_theoretical[raag][ii], 6)})")

In [13]:
raags_and_artists = {
    "Bhoop": {
        "AjoyChakrabarty": "https://www.youtube.com/watch?v=AaSH_AF-Gzg",
        "MilindRaikar": "https://www.youtube.com/watch?v=9a7NhReDWy8",
        "DKDatar": "https://www.youtube.com/watch?v=z5RemO4d41o",
    },
    "Deshkar": {
        "AjoyChakrabarty": "https://www.youtube.com/watch?v=0vseJcm5xPE",
        "KalaRamnath": "https://www.youtube.com/watch?v=DBgqgTrEc1o",
        "PravinGodkhindi": "https://www.youtube.com/watch?v=w-uh1vjXVaU",
    },
    "Yaman" : {
        "KalaRamnath": "https://www.youtube.com/watch?v=kvTqtXP6lmo",
        "NandiniShankar": "https://www.youtube.com/watch?v=ldS89LPpQ_w",
        "DKDatar": "https://www.youtube.com/watch?v=EemtViN7zM8",
    },
}

In [15]:
# ONLY ONCE
# fetch_audio(raags_and_artists, DATA_DIR)

In [17]:
# ONLY ONCE
# for raag, vv in raags_and_artists.items():
#     raag_samples = {}
#     for artist, _ in vv.items():
#         track_mp3 = glob.glob(f"{DATA_DIR}/{raag}-{artist}*.mp3")[0]
#         annotate_tonic(track_mp3[:-4])

## Algorithmic Frequencies

In [11]:
ftracks = {}
for track_mp3 in glob.glob(DATA_DIR + f"*Deshkar*.mp3"):
    track_path = track_mp3[:-4]
    y_sample, sr, ctonic = read_sample_and_tonic(track_path)
    ftrack = pitch.track_pitch_pyin(y_sample, sr, ctonic)
    ftracks[track_path.split("/")[-1]] = ftrack

In [ ]:
for tt, ftrack in ftracks.items():
    plt.figure(figsize=(20, 10))
    plt.plot(ftrack[:1000])
    plt.title(tt)
    plt.show()